## 빅데이터 비즈니스 관점
- 데이터 많은 회사가 결국 이김
- 데이터를 활용
  + OPEN API 형태로 돈을 제공
- API 제공하는 최사는 굉장히 많음
  + 빅테크 기업들은 거의 다 제공 중

In [4]:
import requests
                                                                        # json으로 출력(xml로 변경시 xml로 출력함) 
url = "http://data.ex.co.kr/openapi/trtm/realUnitTrtm?key=1308370912&type=json&iStartUnitCode=101&iEndUnitCode=103&numOfRows=100&pageNo=1"
req = requests.get(url)
print(req)

<Response [200]>


### json 형태로 출력

In [ ]:
json_result = req.json()
json_result

- 데이터 가공

In [ ]:
cars = json_result['realUnitTrtmVO']
# len(cars)
records = []
for car in cars:
  dic = {}
  dic['date'] = car['stdDate']
  dic['time'] = car['stdTime']
  dic['timeAvg'] = car['timeAvg']
  dic['cartype'] = car['tcsCarTypeDivName']
  records.append(dic)

records

In [13]:
import pandas as pd
df = pd.DataFrame(records)
df
# df.to_csv('data.csv')

### XML 활용

In [ ]:
import lxml
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "http://data.ex.co.kr/openapi/trtm/realUnitTrtm?key=1308370912&type=xml&iStartUnitCode=101&iEndUnitCode=103"
result = requests.get(url)
content = result.content
soup = BeautifulSoup(content, "lxml") # html.parser 
# print(soup)
print(soup.prettify())

timeAvg_lists = soup.find_all("timeavg")
stddate_lists = soup.find_all("stddate")
stdtime_lists = soup.find_all("stdtime")

avgTime = []
stddate = []
stdtime = []
for timeAvg, date, time in zip(timeAvg_lists, stddate_lists, stdtime_lists):
  print(timeAvg, date, time)
  # print(time.get_text())
  avgTime.append(timeAvg.get_text())
  stddate.append(date.get_text())
  stdtime.append(time.get_text())

df = pd.DataFrame({
    "date": stddate, 
    "time": stdtime, 
    "avgTime": avgTime
})


## 네이버 주식 데이터 수집
- 다중 페이지


In [ ]:
# !pip install fake_useragent
# 가상 useragent 사용하기 위한 패키지. 사용해도 되고 안해도 되는 패키지.
import requests
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

# 035720 카카오
# 005930 삼성
# 004690 삼천리
company_code = '005930'
url = 'https://finance.naver.com/item/sise_day.naver?code=' + company_code
# print(url)

ua = UserAgent()

custom_headers = {
    'User-agent':ua.ie
    # 'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

response = requests.get(url, headers = custom_headers)
response.status_code

soup = BeautifulSoup(response.content, "html.parser")
# print(soup)

last_page = int(soup.select_one('td.pgRR').a['href'].split('=')[2]) # -1은 맨 끝이니 -1로 대체 가능
# print(last_page)

df = None # 페이지별로 데이터프레임 누적 병합
count = 0 # break문 작성용

for page in range(1, last_page+1):
  # urlp = url + '&page={}'.format(page)
  # print(urlp)
  res = requests.get(f'{url}&page={page}',headers = custom_headers)
  # print(res)
  df = pd.concat([df,pd.read_html(res.text, encoding='euc-kr')[0]], ignore_index=True)

  if count > 2:
    break
  count += 1

df.dropna(inplace=True) # drop na 를 이용해 결측지 제거
df.reset_index(drop=True, inplace=True) 
# 행 인덱스를 재배열 함수 - drop=True : 기존의 인덱스를 버림 / inplace=True : 다른 변수에 입력하지 않고 df에 바로 적용
df

In [106]:
# 증시 기업 코드
company_codes = ["030200", "005930", "068270", "035720"]
company_names = ["KT", "삼성전자", "셀트리온", "카카오"]
url = "https://finance.naver.com/item/main.nhn?code=" + company_code

import requests
from bs4 import BeautifulSoup
import pandas as pd

nD = []

for i in range(0, (len(company_codes))):
  url = "https://finance.naver.com/item/main.nhn?code="+company_codes[i]
  custom_header = {
      'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
  }

  res = requests.get(url,headers = custom_header)
  soup = BeautifulSoup(res.text, "html.parser")

  div = soup.find('div', class_ = 'today')
  # print(div)

  nV = div.find_all('span', class_ = 'blind')[0].get_text()
  nD.append(nV)
  # code = company_codes[i]
  # name = company_names[i]
  # nD = pd.DataFrame({
      # '회사 코드' : [code],
      # '회사 이름' : [name],
      # '현재 가격' : nV
  # })
  # print(nD)
  # df = pd.concat([df,nD], ignore_index=True)
  # df = pd.concat([df,pd.read_html(res.text, encoding='euc-kr')[0]], ignore_index=True)

dic = {
    '회사 코드' : company_codes,
    '회사 이름' : company_names,
    '현재 가격' : nD
}
df = pd.DataFrame(dic)
print(df)

# df.reset_index(drop=True, inplace=True)
# print(df)


    회사 코드 회사 이름    현재 가격
0  030200    KT   34,250
1  005930  삼성전자   60,400
2  068270  셀트리온  165,000
3  035720   카카오   60,700


In [ ]:
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
from datetime import datetime
import time
import pandas as pd

def get_Code(company_code):
    url ="https://finance.naver.com/item/main.nhn?code=" + company_code
    ua = UserAgent()
    headers = { 'User-agent': ua.ie }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup
 
def get_Price(company_code):
    soup = get_Code(company_code)
    no_today = soup.select_one('p.no_today')
    print(no_today)
    blind = no_today.select_one('span.blind')
    return blind.text

def create_df(company_code, company_name, prices_list):
  data = pd.DataFrame({"종목코드": company_code, 
                       "상장회사": company_name, 
                       "주가": prices_list})
  return data
 
# 증시 기업코드
company_codes = ["030200", "005930", "068270", "035720"]
company_names = ["KT", "삼성전자", "셀트리온", "카카오"]

if __name__ == '__main__':
    now = datetime.now()
    print("-" * 60)
    print(now)
    print("-" * 60)
    
    prices_list = []
    for elm in company_codes:
        nowPrice = get_Price(elm)
        prices_list.append(nowPrice)

    # print(get_prices)

    stock_data = create_df(company_codes, company_names, prices_list)
    print(stock_data)

    print("-" * 60)

# Selenium 방식 https://dschloe.github.io/python/crawling/python_selenium_crawling/